## 读取地震数据格式（seed、mseed等）

In [ ]:
import obspy
from obspy.core import UTCDateTime
from preprocess import *
import numpy as np
import h5py

### 读取数据
- **obspy**可以将各种格式的地震数据（例如SAC，MiniSEED，GSE2，SEISAN，Q等）使用**read（）**函数的对象导入到**Stream**对象中。
- **Stream**类似一个包含多个**Trace**对象（例如无间隙连续时间序列和相关的对象头/元信息）的列表。
+ 每个**Trace**对象都有一个名为**data**的属性，对应实际时间序列的NumPy ndarray，以及**stats**属性（Stats对象是一个包含所有元信息的字典）。
- **Stats**中的starttime和endtime也都是UTCDateTime对象。
```
>>> import obspy
>>> st = obspy.read('http://examples.obspy.org/RJOB_061005_072159.mseed')
>>> print(st)
>>> tr = st[0]
>>> print(tr)
```

In [ ]:
st = obspy.read(r"F:\星火\数据\云南盈江地震\20140604.HS.QKT.seed")

In [ ]:
print(st)

In [ ]:
tr = st[0]

In [ ]:
print(tr)

- 在每个**Trace**中可通过**stats**关键词对地震元数据（描述实际波形数据的数据）进行访问
```
>>> print(tr.stats)
```
- 在每个**Trace**中可以通过**data**关键词对实际波形数据进行检索
```
>>> tr.data
array([-38,  12,  -4, ..., -14,  -3,  -9])
>>> tr.data[0:3]
array([-38,  12,  -4])
>>> len(tr)
36000
```
- **Stream**对象提供了一个用于快速预览波形的`plot（）`方法
```
>>> st.plot()
```

In [ ]:
print(tr.stats)

In [ ]:
tr.data

In [ ]:
len(tr)

In [ ]:
st.plot()

### 同一台站多日数据合并
先把台站多日数据读取到一个**Stream**对象中，然后用**merge**方法合并，`fill_value=0`为填充值。
```
>>> st = obspy.read(r"***.seed")
>>> st += obspy.read(r"***.seed")
*
*
*
>>> st.merge(method=1,fill_value=0)
```

### UTCDateTime（世界标准时间数据）
ObsPy中的所有绝对时间值均由UTCDateTime类一致处理。因为精度问题它基于高精度POSIX时间戳，而不是Python datetime类。
```
>>> from obspy.core import UTCDateTime
>>> UTCDateTime("2012-09-07T12:15:00")
UTCDateTime(2012, 9, 7, 12, 15)
>>> UTCDateTime(2012, 9, 7, 12, 15, 0)
UTCDateTime(2012, 9, 7, 12, 15)
>>> UTCDateTime(1347020100.0)
UTCDateTime(2012, 9, 7, 12, 15)
```
处理时间差
```
>>> time = UTCDateTime("2012-09-07T12:15:00")
>>> print(time + 3600)
2012-09-07T13:15:00.000000Z
>>> time2 = UTCDateTime(2012, 1, 1)
>>> print(time - time2)
21644100.0
```

### 波形绘制：根据给定的时间戳绘制波形图
```
>>> dt = UTCDateTime(2014, 5, 23, 18, 30, 0)
>>> et = UTCDateTime(2014, 5, 23, 19, 40, 0)
>>> st.plot(color='red', number_of_ticks=7,
                   tick_rotation=5, tick_format='%I:%M %p',
                   starttime=dt, endtime=et)
```

In [ ]:
dt = UTCDateTime(2014, 5, 23, 18, 30, 0)

In [ ]:
et = UTCDateTime(2014, 5, 23, 19, 40, 0)

In [ ]:
st.plot(color='red', number_of_ticks=7,
                   tick_rotation=5, tick_format='%I:%M %p',
                   starttime=dt, endtime=et)

### 预处理：窗口化、去趋势标准化
#### 该部分用到的部分函数在preprocess.py中
- 窗口化：从连续波形中按特定时长截取地震事件或者背景噪声的窗口，每一个窗口对应一个标签，是模型训练集的一个样本。
- 去趋势标准化：将截取的原始波形进行去趋势和归一化。

主要函数有：

- 根据目录和地震到时输出地震事件波形的初始时间
```
cat = read_catalog_orig(filename,arriving_time)
filename为地震目录，arriving_time为地震到时
```
- 输出背景噪声窗口化的初始时间
```
cat = noise_train_orig()
```
- 根据地震理论到时从波形文件中截取波形，并标准化
```
streams_slice(st,arrival_time)
st为连续波形Stream对象，arrival_time为地震到时
```

In [ ]:
### 根据目录和地震到时输出地震事件波形的起始时间

cat = read_catalog_orig(filename,arriving_time)
# filename为地震目录，arriving_time为地震到时

# 采用时间偏移方法的数据扩增
# cat = read_catalog_ex4(filename,arriving_time)

In [ ]:
### 输出背景噪声窗口化的初始时间

cat = noise_train_orig()

# 采用时间偏移方法的数据扩增
# cat = noise_train_ex4()

# 测试集背景噪声窗口化的初始时间
# cat = noise_test()

In [ ]:
### 根据窗口起始时间截取数据，并进行标准化

# 地震目录
catlogname = r'F:\PythonProjects\conv_for_quake\yj_catlog\week1.txt'
# 输出地震事件窗口初始时间列表
RHT_e = read_catalog_orig(catlogname,arriving_time)
# 输出背景噪声窗口初始时间列表
RHT_n = noise_train_orig()

# 地震事件所在的连续波形文件
wavename_rht = r'F:\PythonProjects\conv_for_quake\yj_data\20140524-30.RHT.mseed'
# 背景噪声所在的连续波形文件
noisename_rht = r'F:\PythonProjects\conv_for_quake\yj_data\20140523-24.RHT.mseed'

# 新建用于存放地震事件和背景噪声窗口文件的空表
Data_e = []
Data_n = []

# 根据地震事件初试时间列表和连续波形文件，从波形中截取窗口，并进行标准化
st = read(wavename_rht)

for i in RHT_e:
    data = streams_slice(st,i)
    Data_e.append(data)
    
Data_e = np.array(Data_e)
Data_e.shape

# 根据背景噪声初试时间列表和连续波形文件，从波形中截取窗口，并进行标准化
nt = read(noisename_rht)

for i in RHT_n:
    data = streams_slice(nt,i)
    Data_n.append(data)
    
Data_n = np.array(Data_n)
Data_n.shape


### 数据储存

##### 把截取的窗口化地震事件和背景噪声数据储存到文件中

本版本提供了储存到h5文件的功能，需要安装h5py扩展包

In [ ]:
# 将窗口化数据储存到.h5文件中

with h5py.File(r'RHT.h5','a') as f:
    f.create_dataset('event',data = Data_e)
    f.create_dataset('noise',data = Data_n)

In [ ]:
# 读取.h5文件中的数据

dataset = h5py.File(r'RFT.h5', "r")
event_orig = np.array(dataset["event"][:])
nosie_orig = np.array(dataset["noise"][:])

## 读取压缩的文本格式数据

In [ ]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import h5py

from sklearn import preprocessing

In [ ]:
# path为压缩文件所在文件夹路径
path = r"F:\PythonProjects\wavelets\ZJ"

files= os.listdir(path)
# 通过files可以查看文件夹下压缩文件的文件名
files

In [ ]:
len(files)

### 以下分为两部分功能，一部分为读取数据归一化并存入h5文件，一部分为读取文件并绘图展示
根据不同需要选择执行代码

#### 读取数据、归一化、存入h5文件

In [ ]:
# 读取zip压缩文件中的数据并进行归一化

data = []
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))

for i in np.arange(2,len(files)):
    
    read_hey = zipfile.ZipFile(files[i])

    f1 = read_hey.open(r'ZJ_HL_BHN_0.txt','r')
    f2 = read_hey.open(r'ZJ_HL_BHE_1.txt','r')
    f3 = read_hey.open(r'ZJ_HL_BHZ_2.txt','r')
    
    nn = []
    for line in f1.readlines():
        line = line.decode()
        lines = line.rstrip('\r\n')
        lines = lines.encode()
        nn.append(lines)
    nn = np.array(nn)
    nn = nn.reshape((-1,1))

    ne = []
    for line in f2.readlines():
        line = line.decode()
        lines = line.rstrip('\r\n')
        lines = lines.encode()
        ne.append(lines)
    ne = np.array(ne)
    ne = ne.reshape((-1,1))

    nz = []
    for line in f3.readlines():
        line = line.decode()
        lines = line.rstrip('\r\n')
        lines = lines.encode()
        nz.append(lines)
    nz = np.array(nz)
    nz = nz.reshape((-1,1))
    
    # 进行数据归一化
    nn_minmax = min_max_scaler.fit_transform(nn)
    ne_minmax = min_max_scaler.fit_transform(ne)
    nz_minmax = min_max_scaler.fit_transform(nz)
    
    nn_minmax = nn_minmax.reshape((-1,))
    ne_minmax = ne_minmax.reshape((-1,))
    nz_minmax = nz_minmax.reshape((-1,))
    
    n = np.stack((nn_minmax,ne_minmax,nz_minmax), axis=1)
    
    data.append(n)

DATA = np.stack(data, axis=0)
DATA.shape

In [ ]:
# 把数据存入文件
with h5py.File(r'HL_event.h5','a') as f:
    f.create_dataset('event',data = DATA)

#### 读取文件，并绘图展示

In [ ]:
# 波形绘图
read_hey = zipfile.ZipFile(files[13])

f1 = read_hey.open(r'ZJ_HL_BHN_0.txt','r')
f2 = read_hey.open(r'ZJ_HL_BHE_1.txt','r')
f3 = read_hey.open(r'ZJ_HL_BHZ_2.txt','r')

n1 = []
for line in f1.readlines():
    line = line.decode()
    n1.append(line.rstrip('\r\n'))
n1 = np.array(n1)
n1 = n1.reshape((-1,1))

n2 = []
for line in f2.readlines():
    line = line.decode()
    n2.append(line.rstrip('\r\n'))
n2 = np.array(n2)
n2 = n2.reshape((-1,1))

n3 = []
for line in f3.readlines():
    line = line.decode()
    n3.append(line.rstrip('\r\n'))
n3 = np.array(n3)
n3 = n3.reshape((-1,1))

In [ ]:
# 进行数据归一化
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
n1_minmax = min_max_scaler.fit_transform(n1)
n2_minmax = min_max_scaler.fit_transform(n2)
n3_minmax = min_max_scaler.fit_transform(n3)

In [ ]:
plt.figure(figsize=(16, 8))

plt.subplot(311)
plt.plot(n1_minmax)

plt.subplot(312)
plt.plot(n2_minmax)

plt.subplot(313)
plt.plot(n3_minmax)

plt.show()